In [1]:
!pip install sentencepiece
!wget https://raw.githubusercontent.com/google/sentencepiece/master/data/botchan.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.4 MB/s eta 0:00:00
--2023-04-23 09:42:46--  https://raw.githubusercontent.com/google/sentencepiece/master/data/botchan.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 278779 (272K) [text/plain]
Saving to: ‘botchan.txt’

botchan.txt         100%[===================>] 272.25K  --.-KB/s    in 0.003s  

2023-04-23 09:42:47 (88.5 MB/s) - ‘botchan.txt’ saved [278779/278779]



In [2]:
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 98.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 97.3 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=7cf5bd1aa375efcfdcbf5d420a90aa7276754f2cbfe6849761b9f2707565d784
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence-transformers


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from transformers import pipeline, AutoTokenizer,AutoModel, AutoModelForMaskedLM, AutoModelWithLMHead
import torch
import gensim.downloader as api

In [ ]:
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Add a language modeling head to the model
model = AutoModelForMaskedLM.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relations

In [ ]:
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Add a language modeling head to the model
model = AutoModelForMaskedLM.from_pretrained(model_name)

word_bias = {'gorgeous': 0.85, 'sad': 0.15,'blue':0.15,'formula':0.1,'apple':0.05} #tfidf for user
# word_bias = {'gorgeous': 0.99} #normalized tfidf for user

word_token_ids = tokenizer.convert_tokens_to_ids(list(word_bias.keys())) #encode words

embedding_matrix = model.get_input_embeddings().weight.detach().numpy() #logit from model as numpy

bias = np.zeros((len(embedding_matrix),))
for word, bias_value in word_bias.items():
    bias[tokenizer(word)['input_ids'][1:-1]] = bias_value #create bias value on an empty logit

embedding_matrix[word_token_ids, :] *= 1+bias[word_token_ids].reshape(-1, 1) #add bias to logit from model
embedding_matrix = torch.tensor(embedding_matrix) #convert to torch tensor

model.get_input_embeddings().weight = torch.nn.Parameter(embedding_matrix) #replace weights with new biased weights

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relations

In [ ]:
def generate_text(seed_text, max_length=50, penalty=1.0):
    input_ids = torch.tensor(tokenizer.encode(seed_text)).unsqueeze(0)
    output_ids = model.generate(
        input_ids, 
        max_length=max_length+len(input_ids[0]), 
        repetition_penalty=penalty,
        do_sample=True, 
        top_k=50, 
        top_p=0.95, 
        temperature=1.165
    )
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return output_text

In [ ]:
input_text = "The dress looks "
output_text = generate_text(input_text)
print(output_text)

the dress looks beautiful!!!!!!!!!!!!!!!!!!!! the dress is beautiful!!!!!!!!!!!!!!!!!!!!!!!!!


In [ ]:
embedding_matrix

tensor([[-0.0102, -0.0615, -0.0265,  ..., -0.0199, -0.0372, -0.0098],
        [-0.0117, -0.0600, -0.0323,  ..., -0.0168, -0.0401, -0.0107],
        [-0.0198, -0.0627, -0.0326,  ..., -0.0165, -0.0420, -0.0032],
        ...,
        [-0.0218, -0.0556, -0.0135,  ..., -0.0043, -0.0151, -0.0249],
        [-0.0462, -0.0565, -0.0019,  ...,  0.0157, -0.0139, -0.0095],
        [ 0.0015, -0.0821, -0.0160,  ..., -0.0081, -0.0475,  0.0753]])

In [ ]:
embedding_matrix[word_token_ids, :]

tensor([[-0.0546, -0.0156,  0.0058,  ...,  0.0142, -0.0241, -0.0821],
        [-0.0800,  0.0148, -0.0454,  ...,  0.0035, -0.0441, -0.0144],
        [-0.0493,  0.0045, -0.0848,  ...,  0.0331, -0.0436, -0.0743],
        [-0.0117, -0.0083,  0.0525,  ..., -0.1187, -0.0685,  0.0485],
        [-0.0245, -0.0107, -0.1026,  ...,  0.0303, -0.0852, -0.1283]])

In [ ]:
output_text

'the dress looks gorgeous... gorgeous. very gorgeous...... gorgeous................. gorgeous....................'

Trying out a different way where changing similar words' bias to be a bitless than the word in user corpus

In [14]:
import spacy
import numpy as np

In [ ]:
similar_word_model = api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [11]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Add a language modeling head to the model
model = AutoModelForMaskedLM.from_pretrained(model_name)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relations

In [5]:
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Add a language modeling head to the model
model = AutoModelForMaskedLM.from_pretrained(model_name)
user_words = {'gorgeous': 1.45, 'sad': 1.15,'blue':1.15,'formula':1.1,'apple':1.05} #Assume this is normalized tfidf + 1 top k words
similar_words = {}
n = 50

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relations

In [12]:
for key in user_words.keys(): #reduce similar words' bias, increase user word bias
  word = key
  n_similar_words = similar_word_model.most_similar(word, topn=n)
  arr = []
  for i in n_similar_words:
    doc = nlp(i[0])
    x = i[0]
    for tok in doc:
      x = tok.lemma_
    bias = -(1-i[1])
    if x in user_words.keys():
      continue
    if x in similar_words.keys():
      similar_words[x] = min(similar_words[x],bias)
    else:
      similar_words[x] = bias
    arr.append(similar_words[x])
  user_words[key] = 2+max(arr)

In [15]:
user_words.update(similar_words)
word_bias = user_words

word_token_ids = tokenizer.convert_tokens_to_ids(list(word_bias.keys())) #encode words

embedding_matrix = model.get_input_embeddings().weight.detach().numpy() #logit from model as numpy

bias = np.zeros((len(embedding_matrix),))
for word, bias_value in word_bias.items():
    bias[tokenizer(word)['input_ids'][1:-1]] = bias_value #create bias value on an empty logit

embedding_matrix[word_token_ids, :] += bias[word_token_ids].reshape(-1, 1) #add bias to logit from model
embedding_matrix = torch.tensor(embedding_matrix) #convert to torch tensor

model.get_input_embeddings().weight = torch.nn.Parameter(embedding_matrix) #replace weights with new biased weights

In [16]:
input_text = "The dress looks "
input_ids = torch.tensor(tokenizer.encode(input_text)).unsqueeze(0)
output_ids = model.generate(input_ids, num_return_sequences=1)
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [17]:
output_text

'the dress looks amazing..............'

Another way to find similar words not working bcs of memory issue

In [ ]:
"""embedding_matrix = model.get_input_embeddings().weight.detach().numpy()
word_id = tokenizer('gorgeous')['input_ids'][1:-1]
bias = np.zeros((len(embedding_matrix),))
cosine_similarities = np.dot(embedding_matrix, embedding_matrix[word_id].T) / (np.linalg.norm(embedding_matrix, axis=1) * np.linalg.norm(embedding_matrix[word_id]))
similar_word_indices = np.argsort(-cosine_similarities)
"""

Changing relative bias different

In [4]:
similar_word_model = api.load("word2vec-google-news-300")
nlp = spacy.load("en_core_web_sm")

[=================================================-] 99.8% 1660.2/1662.8MB downloaded


NameError: ignored

In [ ]:
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Add a language modeling head to the model
model = AutoModelForMaskedLM.from_pretrained(model_name)


In [ ]:
n_similar_words = similar_word_model.most_similar('happy', topn=n)
m = np.inf
max_X = 0
for i in n_similar_words:
  doc = nlp(i[0])
  x = i[0]
  for tok in doc:
    x = tok.lemma_
  x_id = tokenizer(x)['input_ids'][1:-1]
  if np.linalg.norm(embedding_matrix[x_id]) <= m:
    max_X = x_id
    m = np.linalg.norm(embedding_matrix[x_id])
tokenizer.decode(max_X, skip_special_tokens=True), m

('good', 1.0028118)

In [ ]:
embedding_matrix = model.get_input_embeddings().weight.detach().numpy()
word_id = tokenizer('gorgeous')['input_ids'][1:-1]
#embedding_matrix[word_id] = embedding_matrix[tokenizer('beautiful')['input_ids'][1:-1]]
embedding_matrix[word_id] *= 1+0.88

In [ ]:
input_text = "The dress looks "
input_ids = torch.tensor(tokenizer.encode(input_text)).unsqueeze(0)
output_ids = model.generate(input_ids, num_return_sequences=1)
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(output_text)

the dress looks gorgeous.. gorgeous.... gorgeous..... gorgeous


In [ ]:
user_words = {'gorgeous': 0.35, 'sad': 0.2,'blue':0.15,'formula':0.15,'apple':0.1} #Assume this is normalized tfidf top k words
n = 50

In [ ]:
for key in user_words.keys(): #increase user words bias with max embedding magnitude of similar words
  similar_words = {}
  word = key
  n_similar_words = similar_word_model.most_similar(word, topn=n)
  arr = []
  for i in n_similar_words:
    doc = nlp(i[0])
    x = i[0]
    for tok in doc:
      x = tok.lemma_
    bias = -(1-i[1])
    if x in user_words.keys():
      continue
    if x in similar_words.keys():
      similar_words[x] = min(similar_words[x],bias)
    else:
      similar_words[x] = bias
    arr.append(similar_words[x])
  user_words[key] = 2+max(arr)

Testing relative word approach on T2T model

In [ ]:
similar_word_model = api.load("word2vec-google-news-300")
nlp = spacy.load("en_core_web_sm")

[=================================================-] 100.0% 1662.2/1662.8MB downloaded


In [ ]:
user_words = {'hot': 0.45, 'sad': 0.15,'blue':0.15,'formula':0.1,'apple':0.05} #normalized tf-idf
similar_words = {}
n = 50

In [ ]:
model_name = "mrm8488/t5-base-finetuned-common_gen"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelWithLMHead.from_pretrained(model_name)
embedding_matrix = model.get_input_embeddings().weight.detach().numpy()

In [ ]:
for key in user_words.keys(): #reduce similar words' bias, increase user word bias
  word = key
  n_similar_words = similar_word_model.most_similar(word, topn=n)
  arr = []
  for i in n_similar_words:
    doc = nlp(i[0])
    x = i[0]
    for tok in doc:
      x = tok.lemma_
    bias = -(1-i[1])
    if x in user_words.keys():
      continue
    if x in similar_words.keys():
      similar_words[x] = min(similar_words[x],bias)
    else:
      similar_words[x] = bias
    arr.append(similar_words[x])
  user_words[key] = 2+max(arr)
user_words.update(similar_words)

In [ ]:

word_bias = user_words
bias = np.zeros((len(embedding_matrix),))
for word, bias_value in word_bias.items():
    bias[tokenizer(word)['input_ids'][1:-1]] = bias_value #create bias value on an empty logit

word_token_ids = tokenizer.convert_tokens_to_ids(list(word_bias.keys()))
embedding_matrix[word_token_ids, :] *= 1+bias[word_token_ids].reshape(-1, 1) #add bias to logit from model
embedding_matrix = torch.tensor(embedding_matrix) #convert to torch tensor

model.set_input_embeddings(torch.nn.Embedding.from_pretrained(embedding_matrix))

In [ ]:
input_text = "The beer was"
input_ids = tokenizer.encode(input_text, return_tensors="pt")
output_ids = model.generate(input_ids)
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(output_text)

The beer was a little stale and the beer was a bit stale


In [ ]:
input_text = "She was the "

In [ ]:
input_ids = tokenizer.encode(input_text, return_tensors="pt")
output_ids = model.generate(input_ids)
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(output_text)

a woman was the only woman to ever be rescued.


Attention based trial

In [ ]:
model_name = "mrm8488/t5-base-finetuned-common_gen"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelWithLMHead.from_pretrained(model_name)

/usr/local/lib/python3.9/dist-packages/transformers/models/auto/modeling_auto.py:1322: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [ ]:
word_bias = {'extraordinary': 0.45, 'sad': 0.15,'blue':0.15,'formula':0.1,'apple':0.05} #normalized tf-idf
similar_words = {}
n = 50

In [ ]:
embedding_matrix = model.get_input_embeddings().weight.detach().numpy()

In [ ]:
embedding_matrix.shape

(32128, 768)

Simple approach of replacing tokens

In [ ]:
from sentence_transformers import SentenceTransformer
device = torch.device('cpu')

sentence_transformer_model = SentenceTransformer('bert-base-nli-mean-tokens')
sentence_transformer_model.to(device)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [ ]:
input_text = "The dress looks beautiful and"

In [ ]:
word_bias = {'cute': 0.45, 'sad': 0.2,'blue':0.15,'formula':0.15,'apple':0.05}
user_words = list(word_bias.keys())

In [ ]:
model_name = "mrm8488/t5-base-finetuned-common_gen"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelWithLMHead.from_pretrained(model_name)

/usr/local/lib/python3.9/dist-packages/transformers/models/auto/modeling_auto.py:1322: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [ ]:
input_ids = tokenizer.encode(input_text, return_tensors="pt")
output_ids = model.generate(input_ids)

In [ ]:
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
output_text

'the dress looks beautiful and i love the lace.'

In [ ]:
s = ""
for word in output_text.split():
  max = -1
  max_w = ''
  for word2 in user_words:
    y_true = [word]
    y_pred = [word2]
    sentence_embeddings = sentence_transformer_model.encode(y_true)
    sentence_embeddings2 = sentence_transformer_model.encode(y_pred)
    cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
    output = torch.mean(cos(torch.tensor(sentence_embeddings), torch.tensor(sentence_embeddings2))).item()
    if output>max:
      max = output
      max_w = word2
  if max>0.85:
    s+=" "+max_w
  else:
    s+=" "+word

In [ ]:
s

' the dress looks cute and i love the lace.'

Feeding top-k words into the prompt as condition

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import torch
from transformers import AutoTokenizer, AutoModelWithLMHead
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer, WordNetLemmatizer


In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# set up the T5 tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-common_gen")
model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-common_gen")

In [ ]:
#pre-process docs:

def preprocess(corpus):
  # define stop words
  stop_words = set(stopwords.words('english'))

  # define stemmer and lemmatizer
  stemmer = SnowballStemmer('english')
  lemmatizer = WordNetLemmatizer()
  pre_doc = []
  # pre-process documents
  for doc in corpus:
      # tokenize
      tokens = word_tokenize(doc)
      
      # lowercase
      tokens = [token.lower() for token in tokens]
      
      # remove stop words
      tokens = [token for token in tokens if token not in stop_words]
      
      # stem
      stemmed_tokens = [stemmer.stem(token) for token in tokens]
      
      # lemmatize
      lemmatized_tokens = [lemmatizer.lemmatize(token) for token in stemmed_tokens]
      s = ""
      for word in lemmatized_tokens:
        s += word+" "
      pre_doc.append(s[:-1])
  return pre_doc

# define a function to get the top k words based on their TF-IDF scores
def get_top_k_words(corpus, k):

    corpus = preprocess(corpus)
    vectorizer = TfidfVectorizer()
    tfidf = vectorizer.fit_transform(corpus)

    # Get the top-k words based on their TF-IDF scores
    k = 5
    feature_names = np.array(vectorizer.get_feature_names_out())
    tfidf_sum = tfidf.sum(axis=0).A1  # sum the TF-IDF values across all documents
    top_k_idx = tfidf_sum.argsort()[:-k-1:-1]  # get the indices of the top-k words
    top_k_words = feature_names[top_k_idx]
    return top_k_words

# generate text using the T5 model with the user corpus TF-IDF bias
def generate_text(prompt,corpus, top_k=5, num_return_sequences=1):
    # get the top k words based on TF-IDF scores
    top_k_words = get_top_k_words(corpus, k=top_k)
    # tokenize the prompt and the top k words
    input_text = prompt + ' ' + ' '.join(top_k_words)
    print("input text: ",input_text)
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    # generate text using the T5 model with the user corpus TF-IDF bias
    outputs = model.generate(input_ids)
    # decode the generated text
    decoded_outputs = []
    for output in outputs:
        decoded_output = tokenizer.decode(output, skip_special_tokens=True)
        decoded_outputs.append(decoded_output)
    return decoded_outputs

In [ ]:
# set up the TF-IDF vectorizer ['veganism is cool','i want to be a vegan because hurting animals is bad','meat is murder go vegan','vegan for 3 years']#
corpus = ["she looks hot in the new dress", "he was so absolutely cute today", "she looked absolutely slaying"]#["apple fruit is nutritious", "banana fruit is yellow in color", "orange fruit is a good source of Vitamin C"]#["her dress was an absolute banger", "he was so absolutely cute today", "she looked absolutely stunning in her new dress"]

In [ ]:
prompt = "mall person dress"
generated_text = generate_text(prompt,corpus, top_k=5, num_return_sequences=3)
print(generated_text)